In [ ]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

import sklearn
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, roc_auc_score

import wandb
from wandb.keras import WandbCallback

DATA_ROOT = "/home/gpu1/Desktop/kamilio/Pneumothorax"
MODEL_ROOT = "/home/gpu1/Desktop/kamilio/models"
IMG_SIZE = 256

## Load necessary functions

In [2]:
import albumentations as A
AUTOTUNE = tf.data.AUTOTUNE
from functools import partial

transforms = A.Compose([
    A.HorizontalFlip(0.5),
    A.VerticalFlip(0.5),
    A.ShiftScaleRotate(shift_limit=0.3, scale_limit=0.50, rotate_limit=(-90,90), p=.75)
])

def get_classes(dataset_name):
    #TODO add functionality to pop error if classes in data splits do not match
    class_names = os.listdir(os.path.join(DATA_ROOT, dataset_name, "train"))
    class_names = [int(name) for name in class_names]
    class_names.sort()
    class_names = [str(name) for name in class_names]
    return class_names

def get_classes_dict(dataset_name):
    class_names = get_classes(dataset_name)
    class_names_dict = {}
    for class_name in class_names:
        image_sample_name = os.path.basename(glob.glob(os.path.join(DATA_ROOT, dataset_name, "train", str(class_name),'*'))[0])
        class_names_dict[str(class_name)] = image_sample_name.split('_')[1]
    return class_names_dict

def get_classes_dict_empty_lists(dataset_name):
    class_names = get_classes(dataset_name)
    class_names_dict = {}
    for class_name in class_names:
        class_names_dict[str(class_name)] = []
    return class_names_dict

def get_label(file_path):
    # Convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    # TODO: pass CLASS_NAMES as function parameter
    one_hot = parts[-2] == CLASS_NAMES
    # Integer encode the label
    return int(one_hot)

def decode_img(img):
    # Convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_jpeg(img, channels=3)
    return img

def aug_fn(image):
    data = {"image":image}
    aug_data = transforms(**data)
    aug_img = aug_data["image"]
    return aug_img

def apply_transformations(image):
    aug_img = tf.numpy_function(func=aug_fn, inp=[image], Tout=tf.float32)
    return aug_img

def process_path(file_path, use_augmentations: bool):
    label = get_label(file_path)
    # Load the raw data from the file as a string
    image = tf.io.read_file(file_path)
    image = decode_img(image)
    # Resize and norm the image
    image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
    image = tf.cast(image/255, tf.float32)
    if use_augmentations:
        image = apply_transformations(image)
    return image, label

def load_dataset(dataset_name:str, split_type: str, use_transformations: bool, mini_batch_size: int, shuffle: bool):
    global CLASS_NAMES
    CLASS_NAMES = get_classes(dataset_name)
    dataset = tf.data.Dataset.list_files(f"{DATA_ROOT}/{dataset_name}/{split_type}/*/*", shuffle=False)
    if shuffle:
        dataset = dataset.shuffle(len(dataset), reshuffle_each_iteration=False)
    dataset = dataset.map(partial(process_path, use_augmentations=use_transformations), num_parallel_calls=AUTOTUNE).batch(mini_batch_size)
    return dataset

In [3]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras import layers

def get_initial_weights_type(imagenet_pretrained_backbone: bool):
    if imagenet_pretrained_backbone:
        weights='imagenet'
    else:
        weights=None
    return weights

def rebuild_top(model, class_count: int, top_dropout_rate: float = 0):
    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization(name='batch_normalization')(x)
    if top_dropout_rate != 0:
        x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(class_count, activation="softmax", name="pred")(x)
    return outputs

def build_EffNetB0(class_count: int, imagenet_pretrained_backbone: bool, top_dropout_rate: float = 0):
    
    weights_type = get_initial_weights_type(imagenet_pretrained_backbone)
    
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights=weights_type)
    outputs = rebuild_top(model, class_count, top_dropout_rate)

    model = tf.keras.Model(inputs, outputs, name='EfficientNetB0')
    print("EffNet0 model build successfull")
    
    return model

def build_EffNetB3(class_count: int, imagenet_pretrained_backbone: bool, top_dropout_rate: float = 0):
    
    weights_type = get_initial_weights_type(imagenet_pretrained_backbone)
    
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = EfficientNetB3(include_top=False, input_tensor=inputs, weights=weights_type)
    outputs = rebuild_top(model, class_count, top_dropout_rate)

    model = tf.keras.Model(inputs, outputs, name='EfficientNetB3')
    print("EffNet3 model build successfull")
    
    return model

def build_EffNetB5(class_count: int, imagenet_pretrained_backbone: bool, top_dropout_rate: float = 0):
    
    weights_type = get_initial_weights_type(imagenet_pretrained_backbone)
    
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = EfficientNetB5(include_top=False, input_tensor=inputs, weights=weights_type)
    outputs = rebuild_top(model, class_count, top_dropout_rate)

    model = tf.keras.Model(inputs, outputs, name='EfficientNetB5')
    print("EffNet5 model build successfull")
    
    return model

def build_model(class_count: int, imagenet_pretrained_backbone: bool, architecture: str,  top_dropout_rate: float = 0):
    if architecture == "EffNetB0":
        model = build_EffNetB0(class_count, imagenet_pretrained_backbone, top_dropout_rate)
    elif architecture == "EffNetB3":
        model = build_EffNetB3(class_count, imagenet_pretrained_backbone, top_dropout_rate)
    elif architecture == "EffNetB5":
        model = build_EffNetB5(class_count, imagenet_pretrained_backbone, top_dropout_rate)
    else:
        raise Exception(f"Specified model architecture is not available")
        
    return model
        

In [4]:
class TestCallback(tf.keras.callbacks.Callback):
    def __init__(self, test_ds, predictions_path):
        super().__init__()
        y_true = np.concatenate([y for x, y in test_ds], axis=0)
        self.df = pd.DataFrame({"true": y_true[:,1]})
        if os.path.isfile(predictions_path):
            raise Exception(f"Predictions file was about to be deleted.;(( Be more carefull")

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(test_ds)
        self.df[str(epoch)] = y_pred[:,1]
        
    def on_train_end(self, logs=None):
        self.df.to_csv(predictions_path)

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_loss(hist, title: str, output_file: str):
    best_epoch = np.argmin(hist.history["val_loss"])
    
    sns.set_style("whitegrid")
    plt.plot(hist.history["loss"])
    plt.plot(hist.history["val_loss"])
    plt.axvline(x=best_epoch, color='r', linestyle='--', label="Best epoch")
    plt.annotate(f'Best epoch: {best_epoch}', xy=(best_epoch, hist.history["val_loss"][best_epoch]),
                 xytext=(best_epoch, hist.history["val_loss"][best_epoch] + 0.05),
                 arrowprops=dict(facecolor='black', arrowstyle='->'))
    plt.title(title)
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend(["Train", "Validation", "Best epoch"], loc="upper left")
    plt.savefig(output_file, dpi=300)
    plt.show()


## Visiaulize samples from downstream taskdataset

In [ ]:
dataset_name = "data_v4.0.0"

map_dict = {'0': 'negative', '1': 'positive'}
sample_dict = {'0': [], '1': []}

rot_train_dataset_eg = load_dataset(
                              dataset_name = dataset_name, 
                              split_type = "train", 
                              use_transformations = True, 
                              mini_batch_size = 1,
                              shuffle = True)

iterator=iter(rot_train_dataset_eg)

for _ in range(50):
    sample = next(iterator)
    image = sample[0].numpy().reshape(IMG_SIZE,IMG_SIZE,3)
    label = tf.math.argmax(sample[1], axis=1).numpy()[0]
    sample_dict[str(label)].append(image)

In [ ]:
class_count = len(sample_dict)
plt.figure(figsize=(class_count*5,4))
i=1
for row in range(class_count):
    for column in range(class_count):
        plt.subplot(class_count, class_count*5, i)
        image = sample_dict[str(row)][column]
        classs = map_dict[str(row)]
        plt.imshow(image)
        plt.title(f'{classs}')
        plt.axis('off')
        i += 1

## Fine-tune pretrained model for brain tumor classification task

In [8]:
configA={
     "type": "Downstream",
     "task": "",
     "dataset_origin": "Pneumothorax",
     "dataset_name": "data_v4.0.0",
     "SSL_checkpoint": "041",
     "use_transformations": True,
     "frozen_backbone": False,
     "loss_function": "categorical_crossentropy",
     "top_dropout_rate": 0.2,
     "learning_rate": 0.0001,
     "batch_size": 32,
     "epochs": 1000,
     "early_stop_patience": 200}

configB={
     "type": "Downstream",
     "task": "",
     "dataset_origin": "Pneumothorax",
     "dataset_name": "data_v4.0.1",
     "SSL_checkpoint": "041",
     "use_transformations": True,
     "frozen_backbone": False,
     "loss_function": "categorical_crossentropy",
     "top_dropout_rate": 0.2,
     "learning_rate": 0.0001,
     "batch_size": 32,
     "epochs": 1000,
     "early_stop_patience": 200}

configC={
     "type": "Downstream",
     "task": "",
     "dataset_origin": "Pneumothorax",
     "dataset_name": "data_v4.0.2",
     "SSL_checkpoint": "041",
     "use_transformations": True,
     "frozen_backbone": False,
     "loss_function": "categorical_crossentropy",
     "top_dropout_rate": 0.2,
     "learning_rate": 0.0001,
     "batch_size": 32,
     "epochs":1000,
     "early_stop_patience": 200}

configD={
     "type": "Downstream",
     "task": "",
     "dataset_origin": "Pneumothorax",
     "dataset_name": "data_v4.0.3",
     "SSL_checkpoint": "041",
     "use_transformations": True,
     "frozen_backbone": False,
     "loss_function": "categorical_crossentropy",
     "top_dropout_rate": 0.2,
     "learning_rate": 0.0001,
     "batch_size": 32,
     "epochs":1000,
     "early_stop_patience": 200}


configs = {"1": configA, "2": configB, "3": configC, "4": configD}

architectures = {"3": "EffNetB3"}
initial_weigths = {"ImageNet": [], "Pre-text": [3]}

In [ ]:
for major in [10]:    
    for minor in [3]:  
        for micro in [1, 2, 3, 4]:
            for patch in [0]:
                model_name = f"model_v{str(major)}.{str(minor)}.{str(micro)}.{str(patch)}"
                
                run = wandb.init(entity = 'ssl_bakalauras',
                                 project=f"Downstream_v{str(major)}",
                                 name = model_name,
                                 config = configs[str(micro)])
                

                print(configs[str(micro)])
                print(f"\nModel version: {model_name}\n")
                print(f"{architectures[str(minor)]} architecture will be used")
                config = wandb.config
                project_name = run.project
                
                path_to_model = os.path.join(MODEL_ROOT, project_name, model_name)

                tf.keras.backend.clear_session()

                class_names = get_classes(config.dataset_name)
                print(f"\nClasses: {class_names}\n")
                if config.use_transformations:
                    transformations = str(transforms)
                    print(f"Using transfromations for train dataset:\n{transformations}\n")
                else:
                    transformations = None
                    print(f"No transformations will be used")


        # Load datasets

                train_ds = load_dataset(
                                      dataset_name = config.dataset_name, 
                                      split_type = "train", 
                                      use_transformations = config.use_transformations, 
                                      mini_batch_size = config.batch_size,
                                      shuffle = True)

                val_ds = load_dataset(
                                      dataset_name = config.dataset_name, 
                                      split_type = "val", 
                                      use_transformations = False, 
                                      mini_batch_size = config.batch_size,
                                      shuffle = True)

                test_ds = load_dataset(
                                      dataset_name = config.dataset_name, 
                                      split_type = "test", 
                                      use_transformations = False, 
                                      mini_batch_size = config.batch_size,
                                      shuffle = False)
        # Initialize model


                if minor in initial_weigths["ImageNet"] and micro == 0:
                    upstream_model_type = "No_model__from_scratch"
                    pstream_model_version = "None"
                    model = build_model(
                                    class_count = len(class_names), 
                                    imagenet_pretrained_backbone = False,
                                    architecture = architectures[str(minor)],
                                    top_dropout_rate = config.top_dropout_rate)
                    print(f"Successfully initialized {architectures[str(minor)]} model from scratch")
                elif minor in initial_weigths["ImageNet"] and micro !=0:
                    upstream_model_type = f"ImageNet"
                    upstream_model_version = "None"
                    model = build_model(
                                    class_count = len(class_names), 
                                    imagenet_pretrained_backbone = True,
                                    architecture = architectures[str(minor)],
                                    top_dropout_rate = config.top_dropout_rate)
                    print(f"Succesfully loaded weights of ImageNet pretrained {architectures[str(minor)]} model as initial weights")
                elif minor in initial_weigths["Pre-text"]:
                    upstream_model_type = "SSL"
                    upstream_model_version = f"model_v{str(major)}.{str(minor)}.0"
                    path_to_upstream_model = os.path.join(MODEL_ROOT, f"SSL_v{str(major)}",upstream_model_version)
                    model = tf.keras.models.load_model(path_to_upstream_model)
                    print(f"Succesfully loaded weights of SSL {upstream_model_version} model")
                    if config.SSL_checkpoint:
                        model.load_weights(os.path.join(path_to_upstream_model, 'checkpoints', f"weights00000{str(config.SSL_checkpoint)}.ckpt"))
                        print(f"Succesfully loaded SSL checkpoint of epoch: {str(config.SSL_checkpoint)}")
                    # Reinitializing dense classification layer
                    inputs = model.input
                    x = model.layers[-5].output
                    x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
                    x = layers.BatchNormalization(name='batch_normalization')(x)
                    top_dropout_rate = config.top_dropout_rate
                    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
                    outputs = layers.Dense(len(class_names), activation="softmax", name="pred")(x)

                    model = tf.keras.Model(inputs, outputs, name=architectures[str(minor)])
                    print(f"Succesfully changed model head")
                else:
                    raise Exception("Failed to load upstream model")

                wandb.log({
                    ""
                    "architecture": architectures[str(minor)],
                    "class_names": class_names,
                    "transformations": transformations,
                    "upstream_model_type": upstream_model_type,
                    "upstream_model_version":upstream_model_version
                })

                # Freezing the Convolutional Layers while keeping Dense layers as Trainable
                if config.frozen_backbone:
                    for layer in model.layers:
                        if not (layer.name in ['pred', 'batch_normalization']):
                            layer.trainable=False
                        else:
                            layer.trainable=True
                    print("Model backbone layers were frozen successfully\n")
                else:
                    print("None of the model layers are frozen\n")

        # Compile model
                optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)
                model.compile(optimizer=optimizer, loss=config.loss_function, metrics = ['acc', tf.keras.metrics.AUC()])

        # Train the model
                predictions_path = os.path.join(DATA_ROOT, "test_ds_predictions", f'{model_name}-{str(config.early_stop_patience)}.csv')
                model.summary()
                history = model.fit(train_ds,
                          epochs=config.epochs,
                          validation_data=val_ds,
                          callbacks=[WandbCallback(save_model=(False)),
                                     TestCallback(test_ds, predictions_path),
                                     tf.keras.callbacks.EarlyStopping(patience=config.early_stop_patience, 
                                                                      restore_best_weights=True)])
                
                plot_loss(history, "Downstream model loss graphs",  f"{path_to_model}_seaborn.png")

        # Test the model 

                test_loss, test_acc, _ = model.evaluate(test_ds)
                y_true = np.concatenate([y for x, y in test_ds], axis=0)
                y_prob = model.predict(test_ds)
                test_auc = roc_auc_score(y_true, y_prob)


                test_ds = load_dataset(
                          dataset_name = config.dataset_name, 
                          split_type = "test", 
                          use_transformations = False, 
                          mini_batch_size = 1,
                          shuffle=True)

                y_true = []
                y_pred = []
                for sample in test_ds:
                    image = sample[0]
                    y_true.append(tf.math.argmax(sample[1], axis=1).numpy()[0])
                    y_pred.append(tf.math.argmax(model.predict(image), axis=1).numpy()[0])
                print(classification_report(y_true, y_pred))

                wandb.log({
                        "test_loss": test_loss,
                        "test_acc": test_acc,
                        "test_auc": test_auc,
                        "conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_true, preds=y_pred,
                            class_names=class_names)
                })

        # Save the model    

                if minor in initial_weigths["ImageNet"]:
                    model.save_weights(os.path.join(path_to_model, "checkpoint"))
                elif minor in initial_weigths["Pre-text"]:
                    model.save(path_to_model)

                run.finish()
            
            